In [1]:
import cv2 as cv
import mediapipe as mp
import math
import numpy as np

In [2]:
face_mesh = mp.solutions.face_mesh
drawing = mp.solutions.drawing_utils
drawing_styles = mp.solutions.drawing_styles

face_mesh_inst = face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,    
)

In [3]:
def rotation_vector_to_euler(rvec):
    R, _ = cv.Rodrigues(rvec)
    sy = math.sqrt(R[0, 0]**2 + R[1, 0]**2)
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.degrees([x, y, z])  # pitch, yaw, roll

In [4]:
# 3D Model Points from gaze_detector.py
face_3d_model = np.array([
    (0.0, 0.0, 0.0),          # Nose tip (1)
    (0.0, -63.6, -12.5),     # Chin (152)
    (-43.3, 32.7, -26.0),    # Left eye outer corner (33)
    (43.3, 32.7, -26.0),     # Right eye outer corner (263)
    (-28.9, -28.9, -24.1),   # Left mouth corner (61)
    (28.9, -28.9, -24.1)     # Right mouth corner (291)
], dtype=np.float64)

In [ ]:
cap = cv.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success: break

    # Mirror for natural view
    frame = cv.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    results = face_mesh_inst.process(rgb_frame)

    if results.multi_face_landmarks:
        landmarks_list = results.multi_face_landmarks[0]
        landmarks = landmarks_list.landmark
        
        # 1. Draw Full Face Mesh
        drawing.draw_landmarks(
            image=frame,
            landmark_list=landmarks_list,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=drawing_styles.get_default_face_mesh_tesselation_style()
        )

        # 2. Extract 6 key points indices matching backend and draw highlighting
        indices = [1, 152, 33, 263, 61, 291]
        image_points = []
        
        for idx in indices:
            point = landmarks[idx]
            px, py = int(point.x * w), int(point.y * h)
            image_points.append((px, py))
            
            # Visual Feedback: Highlighting the 6 key points on top of the mesh
            cv.circle(frame, (px, py), 4, (0, 0, 255), -1)
            cv.putText(frame, str(idx), (px + 5, py + 5), cv.FONT_HERSHEY_PLAIN, 0.8, (0, 255, 255), 1)

        image_points = np.array(image_points, dtype=np.float64)

        # 3. PnP Calculation
        focal_length = w
        cam_matrix = np.array([[focal_length, 0, w/2], [0, focal_length, h/2], [0, 0, 1]], dtype=np.float64)
        dist_coeffs = np.zeros((4, 1))

        success_pnp, rvec, tvec = cv.solvePnP(face_3d_model, image_points, cam_matrix, dist_coeffs)

        if success_pnp:
            pitch, yaw, roll = rotation_vector_to_euler(rvec)

            # Determine Direction
            direction = "Forward"
            if yaw > 15: direction = "Looking Left"
            elif yaw < -15: direction = "Looking Right"
            elif pitch > 15: direction = "Looking Up"
            elif pitch < -15: direction = "Looking Down"

            # Display Results
            cv.putText(frame, f"Gaze: {direction}", (30, 120), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv.putText(frame, f"P:{pitch:.1f} Y:{yaw:.1f} R:{roll:.1f}", (30, 40), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("Face Mesh & Key Landmarks", frame)
    if cv.waitKey(1) & 0xFF == 27: break

cap.release()
cv.destroyAllWindows()